In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt

C:\Users\User\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
tf.set_random_seed(777)

In [4]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [13]:
# hyper parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [14]:
class Model :
    def __init__(self, sess, name) :
        self.sess = sess
        self.name = name
        self._build_net()
        
    def _build_net(self) :
        with tf.variable_scope(self.name) :
            
            self.training = tf.placeholder(tf.bool)
            
            # input placeholders
            self.X = tf.placeholder(tf.float32, [None, 784])
            X_img = tf.reshape(self.X, [-1, 28, 28, 1])
            self.Y = tf.placeholder(tf.float32, [None, 10])
            
            # Layer1
            conv1 = tf.layers.conv2d(inputs = X_img, filters = 32, 
                                     kernel_size = [3, 3], padding = 'SAME', activation = tf.nn.relu)
            pool1 = tf.layers.max_pooling2d(inputs = conv1, pool_size = [2, 2], padding = 'SAME', strides = 2)
            dropout1 = tf.layers.dropout(inputs = pool1, rate = 0.3, training = self.training)
            
            # Layer2
            conv2 = tf.layers.conv2d(inputs = dropout1, filters = 64, 
                                     kernel_size = [3, 3], padding = 'SAME', activation = tf.nn.relu)
            pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size = [2, 2], padding = 'SAME', strides = 2)
            dropout2 = tf.layers.dropout(inputs = pool2, rate = 0.3, training = self.training)
            
            # Layer3
            conv3 = tf.layers.conv2d(inputs = dropout2, filters = 128, 
                                     kernel_size = [3, 3], padding = 'SAME', activation = tf.nn.relu)
            pool3 = tf.layers.max_pooling2d(inputs = conv3, pool_size = [2, 2], padding = 'SAME', strides = 2)
            dropout3 = tf.layers.dropout(inputs = pool3, rate = 0.3, training = self.training)
            
            # Dense Layer with relu
            flat = tf.reshape(dropout3, [-1, 128 * 4 * 4])
            dense4 = tf.layers.dense(inputs = flat, units = 625, activation = tf.nn.relu)
            dropout4 = tf.layers.dropout(inputs = dense4, rate = 0.5, training = self.training)
            
            # Logits (no activation) Layer
            self.logits = tf.layers.dense(inputs = dropout4, units = 10)
            
        # define cost/loss & optimizer
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = self.logits, labels = self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)
        
        correct_prediction = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
    def train(self, x_data, y_data, training = True) :
        return self.sess.run([self.cost, self.optimizer], feed_dict = {self.X : x_data, self.Y : y_data, self.training : training})
    
    def predict(self, x_test, training = False) :
        return self.sess.run(self.logits, feed_dict = {self.X : x_test, self.training : training})
    
    def get_accuracy(self, x_test, y_test, training = False) :
        return self.sess.run(self.accuracy, feed_dict = {self.X : x_test, self.Y : y_test, self.training : training})

In [19]:
tf.reset_default_graph()

In [20]:
# initialize
sess = tf.Session()
m1 = Model(sess, "m1")

sess.run(tf.global_variables_initializer())

print('Learning Started!')

# train my model
for epoch in range(training_epochs) :
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)
    
    for i in range(total_batch) :
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        c, _ = m1.train(batch_xs, batch_ys)
        avg_cost += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning Finished!')

# Test model and check accuracy
print('Accuracy:', m1.get_accuracy(mnist.test.images, mnist.test.labels))

Learning Started!
Epoch: 0001 cost = 0.282804824
Epoch: 0002 cost = 0.088375010
Epoch: 0003 cost = 0.067006945
Epoch: 0004 cost = 0.056261459
Epoch: 0005 cost = 0.049859039
Epoch: 0006 cost = 0.046684221
Epoch: 0007 cost = 0.041883474
Epoch: 0008 cost = 0.037820286
Epoch: 0009 cost = 0.036539200
Epoch: 0010 cost = 0.033628957
Epoch: 0011 cost = 0.033088955
Epoch: 0012 cost = 0.030121155
Epoch: 0013 cost = 0.027823159
Epoch: 0014 cost = 0.029279240
Epoch: 0015 cost = 0.025311230
Learning Finished!
Accuracy: 0.9936
